In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 200

In [ ]:
def myhist3(image,n_bins):
    interval = 256 / n_bins
    histogram = np.zeros((n_bins,n_bins,n_bins))

    for x in image:
        for y in x:
            histogram[ int(y[0] / interval), int(y[1] / interval), int(y[2] / interval) ] += 1
    histogram = histogram/(len(image.reshape(-1))/3) 
    return histogram

def compare_histogram(hist1,hist2,measure_type):
    # hist1 = hist1.reshape(-1)
    # hist2 = hist2.reshape(-1)
    if measure_type == "l2":
        return np.sqrt(np.sum(np.square(np.subtract(hist1,hist2)))) 
    if measure_type == "chi":
        return 1/2 * np.sum(np.divide( np.square(np.subtract(hist1,hist2)), np.add(np.add(hist1,hist2),1e-10 )))
    if measure_type == "intersection":
        return 1 - np.sum(np.minimum(hist1,hist2))
    if measure_type == "hellinger":
        return np.sqrt( np.multiply( 0.5 ,np.sum(np.square(np.subtract(np.sqrt(hist1), np.sqrt(hist2))))))                            


In [ ]:
i = cv2.imread("./dataset/object_01_1.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)

i2 = cv2.imread("./dataset/object_02_1.png")
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)

i3 = cv2.imread("./dataset/object_03_1.png")
i3 = cv2.cvtColor(i3, cv2.COLOR_BGR2RGB)

h1 = myhist3(i,8)
h2 = myhist3(i2,8)
h3 = myhist3(i3,8)

plt.subplot(3,3,1)
plt.imshow(i)
plt.subplot(3,3,7)
plt.bar(np.arange(len(h1.reshape(-1))), h1.reshape(-1), width=10)

plt.subplot(3,3,2)
plt.imshow(i2)
plt.subplot(3,3,8)
plt.title("L2(H1,H2)=" + str(round(compare_histogram(h1.reshape(-1),h2.reshape(-1),"l2"),5)))
plt.bar(np.arange(len(h2.reshape(-1))), h2.reshape(-1), width=10)

plt.subplot(3,3,3)
plt.imshow(i3)
plt.subplot(3,3,9)
plt.title("L2(H2,H3)=" + str(round(compare_histogram(h1.reshape(-1),h3.reshape(-1),"l2"),5)))
plt.bar(np.arange(len(h3.reshape(-1))), h3.reshape(-1), width=10)

plt.show()

In [ ]:
def get_all(path,n_bins):
    histograms = np.zeros(n_bins*n_bins*n_bins*120)
    histograms = np.reshape(histograms, (120,n_bins*n_bins*n_bins))
    names = []
    index = 0
    for name in os.listdir(path):
        i = cv2.imread(path + name)
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        histograms[index] = np.copy(myhist3(i,n_bins)).reshape(-1)
        names.append(name)
        index += 1
    return [histograms,names]

[histograms,names] = get_all("./dataset/",8)

In [ ]:
main = 19
tip = "hellinger"
w = 10

rankings = np.reshape(np.zeros(120 * 2), (120,2))
for indx, hist in enumerate(histograms):
    rankings[indx,0] = indx
    rankings[indx,1] = compare_histogram( histograms[main], hist, tip)

gr = np.copy(rankings)
rankings = rankings[rankings[:,1].argsort()]

i = cv2.imread("./dataset/" + names[int(rankings[0,0])])
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i2 = cv2.imread("./dataset/" + names[int(rankings[1,0])])
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)
i3 = cv2.imread("./dataset/" + names[int(rankings[2,0])])
i3 = cv2.cvtColor(i3, cv2.COLOR_BGR2RGB)
i4 = cv2.imread("./dataset/" + names[int(rankings[3,0])])
i4 = cv2.cvtColor(i4, cv2.COLOR_BGR2RGB)
i5 = cv2.imread("./dataset/" + names[int(rankings[4,0])])
i5 = cv2.cvtColor(i5, cv2.COLOR_BGR2RGB)
i6 = cv2.imread("./dataset/" + names[int(rankings[5,0])])
i6 = cv2.cvtColor(i6, cv2.COLOR_BGR2RGB)

plt.subplot(2,6,1)
plt.title("slika " + str(int(rankings[0,0])+1))
plt.imshow(i)
plt.subplot(2,6,7)
plt.title(tip + " = " + str(round(rankings[0,1],3)))
plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[0,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,2)
plt.title("slika " + str(int(rankings[1,0])+1))
plt.imshow(i2)
plt.subplot(2,6,8)
plt.title(tip + " = " + str(round(rankings[1,1],3)))
plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[1,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,3)
plt.title("slika " + str(int(rankings[2,0])+1))
plt.imshow(i3)
plt.subplot(2,6,9)
plt.title(tip + " = " + str(round(rankings[2,1],3)))
plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[2,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,4)
plt.title("slika " + str(int(rankings[3,0])+1))
plt.imshow(i4)
plt.subplot(2,6,10)
plt.title(tip + " = " + str(round(rankings[3,1],3)))
plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[3,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,5)
plt.title("slika " + str(int(rankings[4,0])+1))
plt.imshow(i5)
plt.subplot(2,6,11)
plt.title(tip + " = " + str(round(rankings[4,1],3)))
plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[4,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,6)
plt.title("slika " + str(int(rankings[5,0])+1))
plt.imshow(i6)
plt.subplot(2,6,12)
plt.title(tip + " = " + str(round(rankings[5,1],3)))
plt.bar(np.arange(len(histograms[0])), histograms[int(rankings[5,0])], width=w)
plt.tight_layout()

plt.show()

In [ ]:
plt.subplot(1,2,1)
plt.plot(gr[:,1])

grr = np.copy(gr)
grr = grr[grr[:,1].argsort()]
plt.plot(gr[:,1], "ko",markevery=grr[0:5,0].astype("int64"))

plt.subplot(1,2,2)
plt.plot(grr[:,1])
plt.plot(grr[:,1], "ko",markevery=gr[0:5,0].astype("int64"))

plt.show()

In [ ]:
summ = np.divide(sum(histograms),120)

summ = summ.reshape(-1)

plt.subplot(1,2,1)
plt.bar(np.arange(len(summ)), summ, width=3)

lmbd = -6
weights = np.exp(np.multiply( lmbd, summ))
plt.subplot(1,2,2)
plt.bar(np.arange(len(summ)), np.multiply(weights, summ), width=3)
plt.show()

In [ ]:
weighted = np.copy(histograms)

for i in range(120):
    weighted[i] = np.multiply(weighted[i], weights)
    weighted[i] = np.multiply( 1/sum(weighted[i]), weighted[i] )
################################################################
main = 19
tip = "hellinger"
w = 10

rankings = np.reshape(np.zeros(120 * 2), (120,2))
for indx, hist in enumerate(weighted):
    rankings[indx,0] = indx
    rankings[indx,1] = compare_histogram( weighted[main], hist, tip)

gr = np.copy(rankings)
rankings = rankings[rankings[:,1].argsort()]

i = cv2.imread("./dataset/" + names[int(rankings[0,0])])
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i2 = cv2.imread("./dataset/" + names[int(rankings[1,0])])
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)
i3 = cv2.imread("./dataset/" + names[int(rankings[2,0])])
i3 = cv2.cvtColor(i3, cv2.COLOR_BGR2RGB)
i4 = cv2.imread("./dataset/" + names[int(rankings[3,0])])
i4 = cv2.cvtColor(i4, cv2.COLOR_BGR2RGB)
i5 = cv2.imread("./dataset/" + names[int(rankings[4,0])])
i5 = cv2.cvtColor(i5, cv2.COLOR_BGR2RGB)
i6 = cv2.imread("./dataset/" + names[int(rankings[5,0])])
i6 = cv2.cvtColor(i6, cv2.COLOR_BGR2RGB)

plt.subplot(2,6,1)
plt.title("slika " + str(int(rankings[0,0])+1))
plt.imshow(i)
plt.subplot(2,6,7)
plt.title(tip + " = " + str(round(rankings[0,1],3)))
plt.bar(np.arange(len(weighted[0])), weighted[int(rankings[0,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,2)
plt.title("slika " + str(int(rankings[1,0])+1))
plt.imshow(i2)
plt.subplot(2,6,8)
plt.title(tip + " = " + str(round(rankings[1,1],3)))
plt.bar(np.arange(len(weighted[0])), weighted[int(rankings[1,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,3)
plt.title("slika " + str(int(rankings[2,0])+1))
plt.imshow(i3)
plt.subplot(2,6,9)
plt.title(tip + " = " + str(round(rankings[2,1],3)))
plt.bar(np.arange(len(weighted[0])), weighted[int(rankings[2,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,4)
plt.title("slika " + str(int(rankings[3,0])+1))
plt.imshow(i4)
plt.subplot(2,6,10)
plt.title(tip + " = " + str(round(rankings[3,1],3)))
plt.bar(np.arange(len(weighted[0])), weighted[int(rankings[3,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,5)
plt.title("slika " + str(int(rankings[4,0])+1))
plt.imshow(i5)
plt.subplot(2,6,11)
plt.title(tip + " = " + str(round(rankings[4,1],3)))
plt.bar(np.arange(len(weighted[0])), weighted[int(rankings[4,0])], width=w)
plt.tight_layout()

plt.subplot(2,6,6)
plt.title("slika " + str(int(rankings[5,0])+1))
plt.imshow(i6)
plt.subplot(2,6,12)
plt.title(tip + " = " + str(round(rankings[5,1],3)))
plt.bar(np.arange(len(weighted[0])), weighted[int(rankings[5,0])], width=w)
plt.tight_layout()

plt.show()

In [ ]:
# 0 1 1 1 0 0.7 0.5 0.2 0 0 1 0
#[ 0.5, 1.5, 1.8, 1.3, 0.65, 0.95, 0.81, 0.35, 0.06, 0.5, 1, 0.3 ]
#[ 0.3, 1, 0.5 ]
from a2_utils import read_data

def simple_convolution(signal, kernel):
    signal = signal
    if isinstance(signal,str):
        signal = read_data(signal)
    kernel = kernel
    if isinstance(kernel,str):
        kernel = read_data(kernel)
    
    l = int((len(kernel) - 1) / 2)
    result = np.zeros( len(signal) )

    for x in range(l,len(signal)-l):
        result[x] = sum( np.multiply(kernel,signal[x-l:x+l+1]) )
    return [signal,kernel,result]

[signal,kernel,result] = simple_convolution("signal.txt","kernel.txt") 
plt.plot(signal)
plt.plot(kernel) 
plt.plot(result) 

plt.show()

In [ ]:
def simple_convolution2(signal, kernel):
    signal = signal
    if isinstance(signal,str):
        signal = read_data(signal)
    kernel = kernel
    if isinstance(kernel,str):
        kernel = read_data(kernel)
    # signal = [0,0,1,0,0,1,1,3,1,1,6,0,0,6,1,1,3,1,1,0,0,1,0,0]
    l = int((len(kernel) - 1) / 2)
    result = np.zeros( len(signal) )
    newsignal = np.zeros( len(signal) + 2 * l )
    newsignal[l:len(newsignal)-l] = signal

    newsignal[0:l] = np.zeros(l)
    newsignal[len(newsignal)-l:len(newsignal)] = np.zeros(l) 
    # newsignal[0:l] = (np.flip(signal[0:l]))
    # newsignal[len(newsignal)-l:len(newsignal)] = np.flip(signal[len(signal)-l:len(signal)]) 

    for x in range(l,len(newsignal)-l):
        result[x-l] = sum( np.multiply(kernel,newsignal[x-l:x+l+1]) )
    return [signal,kernel,result]

[signal,kernel,result] = simple_convolution2("signal.txt","kernel.txt") 
plt.plot(signal)
plt.plot(kernel) 
plt.plot(result) 
plt.show()


In [ ]:
def gaus_kernel(size, gamma):
    if size > 2 * 3 * gamma + 1:
        size = 2 * 3 * gamma + 1
    gaus = np.arange(-size,size+1)
    gaus = gaus.astype(float)
    gaus[:] = np.multiply(1/(np.sqrt(2 * np.pi) * gamma), np.exp(np.divide(-np.square(gaus[:]),2*np.square(gamma))) )
    return gaus / sum(gaus)

size = 15
gaus1 = gaus_kernel(size,0.5)
gaus2 = gaus_kernel(size,1)
gaus3 = gaus_kernel(size,2)
gaus4 = gaus_kernel(size,3)
gaus5 = gaus_kernel(size,4)
    
plt.plot(np.arange( -int(len(gaus1)/2), int(len(gaus1)/2)+1 ),gaus1, color="cyan")
plt.plot(np.arange( -int(len(gaus2)/2), int(len(gaus2)/2)+1 ),gaus2)
plt.plot(np.arange( -int(len(gaus3)/2), int(len(gaus3)/2)+1 ),gaus3)
plt.plot(np.arange( -int(len(gaus4)/2), int(len(gaus4)/2)+1 ),gaus4)
plt.plot(np.arange( -int(len(gaus5)/2), int(len(gaus5)/2)+1 ),gaus5)
plt.show()

In [ ]:
[signal,_,result1] = simple_convolution2("signal.txt",gaus_kernel(15,2))
[_,_,result1] = simple_convolution2(result1,[0.4,0.6,0.1])

[_,_,result2] = simple_convolution2("signal.txt",[0.4,0.6,0.1])
[_,_,result2] = simple_convolution2(result2,gaus_kernel(15,2))

[_,_,result3] = simple_convolution2( gaus_kernel(15,2), [0.4,0.6,0.1])
result3 = np.flip(result3)
[_,_,result3] = simple_convolution2("signal.txt",result3)

plt.subplot(2,2,1)
plt.plot(signal)
plt.tight_layout()
plt.subplot(2,2,2)
plt.plot(result1)
plt.tight_layout()
plt.subplot(2,2,3)
plt.plot(result2)
plt.tight_layout()
plt.subplot(2,2,4)
plt.plot(result3)
plt.tight_layout()
plt.show()


In [ ]:
from a2_utils import gauss_noise
from a2_utils import sp_noise

def gaussfilter(image,gamma,width):
    k = gaus_kernel(width,gamma)
    return cv2.filter2D(cv2.filter2D(image,-1,cv2.transpose(k),borderType=4),-1,k,borderType=4)

i = cv2.imread("./images/lena.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
igaus = gauss_noise(i,-100)
isp = sp_noise(i,0.1)

plt.subplot(2,2,1)
plt.imshow(igaus,cmap="gray",vmin=0, vmax=255)
plt.subplot(2,2,2)
plt.imshow(isp,cmap="gray",vmin=0, vmax=255)
plt.subplot(2,2,3)
plt.imshow(gaussfilter(igaus,3,3),cmap="gray",vmin=0, vmax=255)
plt.subplot(2,2,4)
plt.imshow(gaussfilter(isp,2,4),cmap="gray",vmin=0, vmax=255)
# plt.tight_layout()
plt.show()

In [ ]:
i = cv2.imread("./images/museum.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)

plt.subplot(1,2,1)
plt.imshow(i,cmap="gray",vmin=0, vmax=255)
plt.subplot(1,2,2)
kernel = np.reshape(np.zeros(9),(3,3))
kernel[1,1] = 2.3
kernel = np.subtract( kernel, np.ones(1) / 8)
plt.imshow(cv2.filter2D(i,-1, kernel),cmap="gray",vmin=0, vmax=255)
plt.show()

In [ ]:
def simple_median(signal, width):
    # l = int((len(kernel) - 1) / 2)
    l = width
    result = np.zeros( len(signal) )
    newsignal = np.zeros( len(signal) + 2 * l )
    newsignal[l:len(newsignal)-l] = signal

    newsignal[0:l] = newsignal[l]
    newsignal[len(newsignal)-l:len(newsignal)] = newsignal[ len(newsignal) - l ]

    for x in range(l,len(newsignal)-l):
        result[x-l] = np.median( newsignal[x-l:x+l+1] )
    return result

def noise(signal, percent):
    new = np.copy(signal)

    for x in range(len(new)):
        if np.random.uniform(0,1) < percent:
            if np.random.uniform(0,1) < 0.5:
                new[x] = 0
            else:
                new[x] = 4
    return new

sig = np.zeros(40)
sig[15:26] = 1
noised = noise(sig,0.1)
median = simple_median(noised,7)
[_,_,g] = simple_convolution2(noised,gaus_kernel(3,1))

plt.subplot(2,2,1)
plt.plot(sig)
plt.ylim([0,4])
plt.subplot(2,2,2)
plt.plot(noised)
plt.ylim([0,4])
plt.subplot(2,2,3)
plt.plot(median)
plt.ylim([0,4])
plt.subplot(2,2,4)
plt.plot(g)
plt.ylim([0,4])
plt.tight_layout()
plt.show()

In [ ]:
def median2d(image,width):
    l = int( width / 2 )
    padded = cv2.copyMakeBorder(np.copy(i),l,l,l,l,4)
    result = np.copy(image)
    for x in range(l,len(image)+l):
        for y in range(l,len(image)+l):
            result[x-l,y-l] = np.median( padded[x-l:x+l+1,y-l:y+l+1] )
    return result

i = cv2.imread("./images/lena.png")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
igaus = gauss_noise(i,-100)
isp = sp_noise(i,0.2)

plt.subplot(2,3,1)
plt.imshow(igaus,cmap="gray",vmin=0, vmax=255)
plt.subplot(2,3,2)
plt.imshow(gaussfilter(igaus,3,3),cmap="gray",vmin=0, vmax=255)
plt.subplot(2,3,3)
plt.imshow(median2d(igaus,7),cmap="gray",vmin=0, vmax=255)

plt.subplot(2,3,4)
plt.imshow(isp,cmap="gray",vmin=0, vmax=255)
plt.subplot(2,3,5)
plt.imshow(gaussfilter(isp,2,4),cmap="gray",vmin=0, vmax=255)
plt.subplot(2,3,6)
plt.imshow(median2d(isp,7),cmap="gray",vmin=0, vmax=255)

plt.tight_layout()
plt.show()

In [ ]:
def laplacian(image, size, gamma):
    if size > 2 * 3 * gamma + 1:
        size = 2 * 3 * gamma + 1
    k = np.zeros( 2 * int(size) + 1)
    k[ int(len(k) / 2) ] = 1
    k = k - gaus_kernel(size,gamma)
    k = k / sum(abs(k))
    k = -k

    r = image[:,:,0].astype(float)
    g = image[:,:,1].astype(float)
    b = image[:,:,2].astype(float)

    return np.dstack(( cv2.filter2D(cv2.filter2D(r,-1,cv2.transpose(k),borderType=4),-1,k,borderType=4), cv2.filter2D(cv2.filter2D(g,-1,cv2.transpose(k),borderType=4),-1,k,borderType=4) , cv2.filter2D(cv2.filter2D(b,-1,cv2.transpose(k),borderType=4),-1,k,borderType=4) ))

i = cv2.imread("./images/cat1.jpg")
i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
i = cv2.flip(i,1)
i2 = cv2.imread("./images/cat2.jpg")
i2 = cv2.cvtColor(i2, cv2.COLOR_BGR2RGB)

l = np.clip(laplacian(i2,14,7), 0, 1)
g = np.clip(gaussfilter(i,27,27) / 255, 0, 1)

plt.subplot(1,3,1)
plt.imshow(l)
plt.subplot(1,3,2)

plt.imshow(g)
plt.subplot(1,3,3)
plt.imshow(l)


plt.imshow( np.clip((l/2 + g/2) , 0, 1 ))
plt.show()
